In [1]:
from omg.datamodule import DataModule
from omg.datamodule.dataloader import OMGTorchDataset
from torch_geometric.loader import DataLoader

In [2]:
import torch

In [3]:
dataset = DataModule.from_lmdb(["./../data/mp_20/train.lmdb"])

example_data = OMGTorchDataset(dataset, convert_to_fractional=True)

loader = DataLoader(example_data, batch_size=10)

2024-09-25 19:33:13.842 | INFO     | omg.datamodule.datamodule:add_from_lmdb:844 - Creating new LMDB file: dataset_274518dcfedeeb30399261097eb4fcb7.lmdb


Adding 27136 configurations to LMDB from ./../data/mp_20/train.lmdb


  0%|          | 0/27136 [00:00<?, ?it/s]

100%|██████████| 27136/27136 [00:00<00:00, 378178.61it/s]


In [4]:
for batch in loader:
    t = torch.rand(1)
    print(t,batch)
    break   

tensor([0.9325]) OMGDataBatch(
  n_atoms=[10],
  batch=[90],
  species=[90],
  cell=[10, 3, 3],
  pos=[90, 3],
  property={},
  ptr=[11]
)


In [5]:
batch.pos

tensor([[ 6.6651e-01,  1.7850e-03,  3.3302e-01],
        [ 1.2300e-04,  9.9972e-01,  2.4600e-04],
        [ 3.3203e-01,  9.9566e-01,  6.6405e-01],
        [ 4.9982e-01,  5.0073e-01,  9.9965e-01],
        [ 1.7374e-01,  4.9349e-01,  3.4747e-01],
        [ 8.3296e-01,  5.0580e-01,  6.6593e-01],
        [ 7.2317e-02,  6.9008e-01,  1.4463e-01],
        [ 4.1868e-01,  6.9828e-01,  8.3736e-01],
        [ 7.4120e-01,  7.1082e-01,  4.8239e-01],
        [ 9.2925e-01,  3.0547e-01,  8.5849e-01],
        [ 2.4877e-01,  2.9142e-01,  4.9754e-01],
        [ 5.8461e-01,  3.0673e-01,  1.6922e-01],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 7.2122e-01,  7.7878e-01,  7.7878e-01],
        [ 5.0000e-01,  2.2122e-01,  7.7878e-01],
        [ 3.4894e-01,  6.5106e-01,  6.5106e-01],
        [ 2.7756e-17,  3.4894e-01,  6.5106e-01],
        [ 5.5511e-17,  6.5106e-01,  3.4894e-01],
        [ 6.5106e-01,  3.4894e-01,  3.4894e-01],
        [ 2.7878e-01,  2.2122e-01,  2.2122e-01],
        [ 5.0000e-01

In [5]:
t = t.repeat(10,1)
t

tensor([[0.7945],
        [0.7945],
        [0.7945],
        [0.7945],
        [0.7945],
        [0.7945],
        [0.7945],
        [0.7945],
        [0.7945],
        [0.7945]])

In [6]:
t.dtype

torch.float32

In [7]:
batch.cell = batch.cell.reshape(10,3,3)


In [8]:
batch.cell.shape

torch.Size([10, 3, 3])

In [9]:
#batch.pos = torch.matmul(batch.pos, torch.inverse(batch.cell))

In [10]:
from omg.model.encoders.cspnet_full import CSPNet_Full

/home/pawanprakash/miniconda3/envs/fermat/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
model = CSPNet_Full(latent_dim=1)
model = model.double()

In [12]:
md = model(batch,t = t)

torch.Size([10, 3, 3])
torch.Size([10, 3, 3])
torch.Size([10, 3, 3])
here


In [15]:
md


Data(species_b=[90, 100], species_eta=[90, 100], pos_b=[90, 3], pos_eta=[90, 3], cell_b=[10, 3, 3], cell_eta=[10, 3, 3])

In [22]:
batch.species

tensor([11, 11, 11, 25, 27, 28,  8,  8,  8,  8,  8,  8, 60, 13, 13, 13, 13, 13,
        13, 13, 13, 29, 29, 29, 29, 48, 49, 49, 49, 52, 52, 52, 52, 33, 44, 45,
        45, 45, 45, 23, 23, 23, 23,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9,  9,
         9, 70, 50, 56, 56, 48, 55, 37, 37, 69, 17, 17, 17, 17, 17, 17, 67, 14,
        28, 28, 28, 28, 19, 19, 37, 69, 23, 23,  8,  8,  8,  8,  8,  8,  8,  8])

In [17]:
md.species_b

tensor([[-0.8398,  0.7658, -0.7543,  ...,  0.4989,  0.2560,  0.4430],
        [-0.8380,  0.7662, -0.7498,  ...,  0.4955,  0.2609,  0.4403],
        [-0.8330,  0.7642, -0.7513,  ...,  0.4981,  0.2632,  0.4423],
        ...,
        [-0.5885,  0.2447, -0.9283,  ..., -0.9191, -0.6786,  1.5135],
        [-0.5950,  0.2425, -0.9271,  ..., -0.9158, -0.6754,  1.5115],
        [-0.5896,  0.2458, -0.9260,  ..., -0.9212, -0.6766,  1.5143]],
       dtype=torch.float64, grad_fn=<AddmmBackward0>)